In [4]:
import re

# 3rd part imports
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

from dashboard.logic.io import GSHEETS_URL, read_gsheet, comment_button, total_assets, \
    total_value_to_num, getDataFrames, findRowColRegex

ImportError: cannot import name 'getDataFrames' from 'dashboard.logic.io' (/home/tonu/Documents/apps/dashboard_invest/dashboard/logic/io.py)

In [8]:
from dashboard.logic.io import get_sheet_names, GSHEETS_URL

get_sheet_names(GSHEETS_URL)

['Investments',
 'Investment Allocation Examples',
 '2023 Forecasts & Risks',
 'Stocks Watchlist',
 'Metals',
 'BizRE',
 'Stocks',
 'Value Investing Ratios',
 'Crypto Consolidation',
 'Crypto INFO (SaleHODL Notes)',
 'X-Test Crypto']

In [3]:
# Read in summary DF and drop empty rows
df = read_gsheet(
    GSHEETS_URL, 
    header=None
)

df_dict = getDataFrames(df)
df_dict.keys()

dict_keys(['main', 'ads', 'announcements', 'advice', 'error_warning', 'risk', 'historical', 'cash_pos', 'general_notes', 'success'])

In [41]:
df1 = df.copy()
df1 = df1.astype(str)

x1 = df1.apply(lambda x: x.str.contains("Property Purchase 23rd")).any(axis='columns').idxmax()

df1.apply(lambda x: x.str.contains("Property Purchase 23rd")).idxmax()

# for idx,row in df1.iloc[x1:].iterrows():
#     if (row == 'nan').all():
#         print(idx)
#         break

references_dict = {
    'main' : ("contains", "Monthly Income", 'up'),
    'ads' : ("contains", 'My Finance Course', 'down'),
    'announce' : ("contains", "Jul 2022: I'm", 'up'),
    'advice' : ("contains", '3x Excellent', 'up'),
    'warning_msg' : ("contains", 'NOTE: Occasionally', 'one'),
    'risk' : ("equals", "RISK", 'down'),
    'historical' : ("equals", "My Historical Investments", "down"),
    'cash_pos' : ("contains", "CASH POSITION", 'one'),
    'general_notes' : ("equals", "GENERAL NOTES", "down"),
    'success' : ("equals", "Investment Success:", "down")
}


def findRefRowCol(df: pd.DataFrame, pattern: str, method: str='contains') -> tuple[int, int]:
    """Find reference row and column numeric index values.

    Args:
        pattern (str): Character sequence or regular expression.
        method (str, optional): Finding reference via pattern within the text ('contains')
            or equalling the value exactly ('equals'). Defaults to 'contains'.

    Raises:
        ValueError: Raises error if other than ['contains', 'equals'] is specified for the method.

    Returns:
        tuple[int, int]: Tuple of ['row_i', 'col_i']
    """
    
    
    # validate that method is correctly entered
    if method not in ['contains', 'equals']:
        raise ValueError(f"{method} can take only values: 'contains' or 'equals'!")
    
    idx_series = None
    if method == 'contains':
        idx_series = df.apply(lambda x: x.str.contains(pattern)).idxmax()
    if method == 'equals':
        idx_series = (df == pattern).idxmax()
    
    return idx_series.max(), idx_series.idxmax() 
    

def sliceDF(df: pd.DataFrame, 
            row_idx1: int=None, 
            row_idx2:int=None, 
            col_idx1: int=None, 
            col_idx2: int=None,
            col_0: bool=True, 
            direction='down') -> pd.DataFrame:
    """Slice DF till the first occuring empty row in given direction.

    Args:
        col_0 (bool, optional): If col_idx1 is actually first column. Defaults to True.
        direction (str, optional): Slice upwards or downwards from given row index. Defaults to 'down'.

    Raises:
        ValueError: If no row indices are specified.

    Returns:
        pd.DataFrame
    """
    
    # assert that at least one of the row indices is specified
    if row_idx1 is None and row_idx2 is None:
        raise ValueError(f"Both row indices can't equal {None}!") 
    
    # if column index is not the first column then None
    col_idx1 = col_idx1 if col_0 is True else None
    
    # find missing row index
    if direction == 'down':
        nan_mask = (df.iloc[row_idx1:,col_idx1:col_idx2] == 'nan').all(axis='columns')
        row_idx2 = None if nan_mask.sum() == 0 else nan_mask.idxmax()
        
    
    if direction == 'up':
        nan_mask = (df.iloc[:row_idx2,col_idx1:col_idx2] == 'nan').all(axis='columns')
        row_idx1 = None if nan_mask.sum() == 0 else nan_mask[::-1].idxmax() + 1
        row_idx2 += 1
         
    return df.iloc[row_idx1:row_idx2,col_idx1:col_idx2]


def getDFs(df: pd.DataFrame, references_dict: dict[str, tuple[str,str,str]]) -> dict[str, pd.DataFrame]:
    """Get subset DFs from a bigger DF based on reference strings.

    Args:
        df (pd.DataFrame): Raw initial DF.
        references_dict (dict): Dictionary of df_name: tuple('method', 'string' , 'direction').
            method has 2 options ['contains' and 'equals'], 'direction' has 3 options 
            ['one', 'down', 'up'] where one means only one line needs to be parsed, 'up' and 'down'
            respectively correspond to the parsing direction from the reference point.
            
            Returns:
        dict: Dictionary of {'df_name': df}
    """
    
    
    # set all cols str type, NaN -> 'nan'
    df = df.reset_index(drop=True).astype(str)
    
    # find reference indices
    dfs = {}
    for k,v in references_dict.items():
        
        row_i, col_i = findRefRowCol(df, pattern=v[1], method=v[0])
        col_i = 0 if k=='risk' else col_i
        
        if v[2] == 'one':
            dfs[k] = df.iloc[row_i,col_i:]
        if v[2] == 'down':
            dfs[k] = sliceDF(df, row_idx1=row_i, col_idx1=col_i, direction=v[2])
        if v[2] == 'up':
            dfs[k] = sliceDF(df, row_idx2=row_i, col_idx1=col_i, direction=v[2])

    # strip all NaN cols and rows
    dfs_clean = {}
    for k,df_ in dfs.items():
        
        if isinstance(df_, pd.Series):
            dfs_clean[k] = (df_
            .replace({'nan': np.nan})
            .dropna(how='any', axis='rows')
            )
        else:
            dfs_clean[k] = (df_
                .replace({'nan': np.nan})
                .dropna(how='all', axis='columns')
                .dropna(how='all', axis='rows')
            )
    
    return dfs_clean

dfs = getDFs(df, references_dict)

In [42]:
dfs.keys()

dict_keys(['main', 'ads', 'announce', 'advice', 'warning_msg', 'risk', 'historical', 'cash_pos', 'general_notes', 'success'])

In [65]:
df_ads = dfs['ads'].copy()
df_ads.columns = ['text', 'hyperlink']
df_ads['icon'] = df_ads.text.str[:1]
df_ads['text'] = df_ads.text.str[2:]
headers = ['Course', 'Mentoring', 'PM Global', 'PM USA', 'PM UK', 'Crypto Security', 'Stock Platform', 'Bank Account']
df_ads['header'] = headers

for i,row in df_ads.iterrows():
    print()

,text,hyperlink,icon,header
1,My Finance Course,https://bit.ly/3qzW4ZB,📈,Course
2,Private 1-on-1 Sessions With Me: (1 hour slots),https://bit.ly/3bxv3xN,🧠,Mentoring
3,Where I Buy Allocated Silver/Gold Globally:,https://bit.ly/3kbgO61,🪙,PM Global
4,Where I Buy Physical Silver/Gold in the USA:,https://bit.ly/3drNR2F,🇺,PM USA
5,Where I Buy Physical Silver/Gold in the UK:,https://bit.ly/3SFvp9E,🇬,PM UK
6,How I Protect My Crypto from hackers:,https://bit.ly/3bK8oyL,🔑,Crypto Security
7,The Stock Platform I use for Europe (FREE Cred...,https://bit.ly/3KoufvK,📈,Stock Platform
8,A Global bank account (that I use) that offers...,https://bit.ly/3JpWOZK,🏦,Bank Account


In [29]:
(dfs['warning_msg']
 .replace({'nan':np.nan})
 .dropna(how='all', axis='columns')
 .dropna(how='all', axis='rows'))

ValueError: No axis named columns for object type Series

# Stocks Watchlist

In [7]:
STOCKS_WATCH_URL = "https://docs.google.com/spreadsheets/d/12-GISr1efphjtpuJLCfQzI2akNXxaJ1iabsG24ib71c/edit#gid=845083323"

# Read in summary DF and drop empty rows
df = read_gsheet(
    STOCKS_WATCH_URL, 
    header=None
)

In [8]:
# find df header row index using regex pattern
header_idx = df.apply(lambda x: x.str.contains("Neil's Value", case=False)).any(axis='columns').argmax()

# separate disclaimer and df
df_disclaimer = df.iloc[:header_idx-1, 0]
df_watch = df.iloc[header_idx:,]

# set first row as header & reset row idxs
df_watch.columns = df_watch.iloc[0].values
df_watch = df_watch.iloc[1:].reset_index(drop=True)

# Generate buttons for 'Notes' column
df_watch.Notes[df_watch.Notes.notna()] = df_watch.Notes[df_watch.Notes.notna()].apply(comment_button)
df_watch = df_watch.fillna("")

# Color Ratings based on category
rating_colormap = {'Sig Undervalued':'green', 'Mod Undervalued':'blue', 'Fair Value':'grey', 'Value Trap?':'red'}
#df_watch["rating_color"] = df_watch.Rating.map(rating_colormap)


In [30]:
df1 = df_disclaimer.copy()

# add period end of the sentence if not so
df1 = df1.to_frame().rename(columns={0: 'info'})
df1.loc[:,'info'] = [x + '.' if x[-1] != '.' else x for x in df1['info']]

df_disc = df1.copy()

df_disc['color'] = ['warning', 'success', 'warning', 'success', 'success', 'danger']
df_disc['color'] = pd.Categorical(df_disc['color'],
                                  categories=['success', 'warning', 'danger'],
                                  ordered=True)
icon_dict = {'warning': 'exclamation-triangle-fill', 'success': 'check-lg', 'danger':'exclamation-octagon-fill'}
df_disc['icon_id'] = df_disc['color'].map(icon_dict)
df_disc = df_disc.sort_values('color')



,info,color,icon_id
1,Please make your own copy of the stocks sectio...,success,check-lg
3,These stocks were found using the techniques t...,success,check-lg
4,"Neil's Stocks Strategy: 80% into Index funds, ...",success,check-lg
0,These Valuations are for the purpose of Long T...,warning,exclamation-triangle-fill
2,NOTE! Many of the metrics below have to be man...,warning,exclamation-triangle-fill
5,"AT THIS TIME, I AM OUT OF ALL STOCKS. PLEASE D...",danger,exclamation-octagon-fill
